In [2]:
#Load libraries
import scipy.io
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

In [4]:
hyp01 = scipy.io.loadmat('hyp1.mat')
hyp02 = scipy.io.loadmat('hyp2.mat')
hyp03 = scipy.io.loadmat('hyp3.mat')
hyp04 = scipy.io.loadmat('hyp4.mat')
hyp05 = scipy.io.loadmat('hyp5.mat')
hyp06 = scipy.io.loadmat('hyp6.mat')
hyp07 = scipy.io.loadmat('hyp7.mat')
hyp08 = scipy.io.loadmat('hyp8.mat')
hyp09 = scipy.io.loadmat('hyp9.mat')
hyp10 = scipy.io.loadmat('hyp10.mat')
feat = scipy.io.loadmat('feat')
hyp01 = hyp01['hyp1']
hyp02 = hyp02['hyp2']
hyp03 = hyp03['hyp3']
hyp04 = hyp04['hyp4']
hyp05 = hyp05['hyp5']
hyp06 = hyp06['hyp6']
hyp07 = hyp07['hyp7']
hyp08 = hyp08['hyp8']
hyp09 = hyp09['hyp9']
hyp10 = hyp10['hyp10']
feat = feat['featurestrim']
hypt = np.concatenate((hyp01, hyp02, hyp03, hyp04, hyp05, hyp06, hyp07, hyp08, hyp09, hyp10), axis = 0)
hypt.shape

mat = np.concatenate((hypt, feat), axis = 1)
df = pd.DataFrame(mat, columns = ['epoch', 'stage', 'delta', 'theta', 'alpha', 'beta1', 'beta2', 'spindle', 'K_complex'])
df.head()

,epoch,stage,delta,theta,alpha,beta1,beta2,spindle,K_complex
0,2397001.0,0.0,-267998884.0,-241505484.0,-336368820.0,-877711454.0,-1.553141e+09,-79331910.0,-39087854.0
1,2400001.0,0.0,-271471318.0,-247198842.0,-327541732.0,-842910362.0,-1.553529e+09,-76485286.0,-39293328.0
2,2403001.0,0.0,-273326486.0,-249414778.0,-330582162.0,-848385458.0,-1.564378e+09,-75834742.0,-39265254.0
3,2406001.0,0.0,-274259644.0,-243100592.0,-332130238.0,-868690592.0,-1.547108e+09,-77460044.0,-39589544.0
4,2409001.0,0.0,-274359650.0,-243435814.0,-327870770.0,-858800212.0,-1.555334e+09,-76668520.0,-39587594.0


In [35]:
#Load Labels
hyp01 = scipy.io.loadmat('SC4001E0-PSG_hyp')
hyp11 = scipy.io.loadmat('SC4011E0-PSG_hyp')
hyp12 = scipy.io.loadmat('SC4012E0-PSG_hyp')
hyp21 = scipy.io.loadmat('SC4021E0-PSG_hyp')
hyp22 = scipy.io.loadmat('SC4022E0-PSG_hyp')
hyp31 = scipy.io.loadmat('SC4031E0-PSG_hyp')
hyp32 = scipy.io.loadmat('SC4032E0-PSG_hyp')
hyp41 = scipy.io.loadmat('SC4041E0-PSG_hyp')
hyp42 = scipy.io.loadmat('SC4042E0-PSG_hyp')
hyp51 = scipy.io.loadmat('SC4051E0-PSG_hyp')

feat = scipy.io.loadmat('feat')
hyp01 = hyp01['hypnogram'][599:1633]
hyp11 = hyp11['hypnogram'][599:1633]
hyp12 = hyp12['hypnogram'][599:1633]
hyp21 = hyp21['hypnogram'][599:1633]
hyp22 = hyp22['hypnogram'][599:1633]
hyp31 = hyp31['hypnogram'][599:1633]
hyp32 = hyp32['hypnogram'][599:1633]
hyp41 = hyp41['hypnogram'][599:1633]
hyp42 = hyp42['hypnogram'][599:1633]
hyp51 = hyp51['hypnogram'][599:1633]
feat = feat['featurestrim']
#feat = feat[]
hypt = np.concatenate((hyp01, hyp11, hyp12, hyp21, hyp22, hyp31, hyp32, hyp41, hyp42, hyp51), axis = 0)
m, n = hypt.shape

#Changing labels to New AASM:
for i in range(m):
    if hypt[i,1]==4:
        hypt[i,1]=3
    elif hypt[i,1]==6:
        hypt[i,1]=0



mat = np.concatenate((hypt, feat), axis = 1)
df = pd.DataFrame(mat, columns = ['epoch', 'stage', 'delta', 'theta', 'alpha', 'beta1', 'beta2', 'spindle', 'K_complex'])
df.head()

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [31]:
def get_score(rf, X_train, X_test, y_train, y_test):
    rf.fit(X_train, y_train)
    return rf.score(X_test, y_test)

In [40]:
features = df.columns[2:9]
kf = KFold(n_splits = 10, shuffle=True)
score = []
for train_index, test_index in kf.split(df[features], df['stage']):
    x_train, x_test = feat[train_index], feat[test_index]
    y_train, y_test = df['stage'][train_index], df['stage'][test_index]
    score.append(get_score(RandomForestClassifier(n_estimators=35), x_train, x_test, y_train, y_test))
    

In [39]:
score

[0.5227817745803357,
 0.5359712230215827,
 0.5155875299760192,
 0.5491606714628298,
 0.5311750599520384,
 0.5227817745803357,
 0.5215827338129496,
 0.5611510791366906,
 0.5347721822541966,
 0.5227817745803357]

In [28]:
df['training?'] = np.random.uniform(0,1,(len(df))) <=.9
#b = ~a
#a.shape
df.head()

,epoch,stage,delta,theta,alpha,beta1,beta2,spindle,K_complex,training?
0,2397001.0,0.0,-267998884.0,-241505484.0,-336368820.0,-877711454.0,-1.553141e+09,-79331910.0,-39087854.0,False
1,2400001.0,0.0,-271471318.0,-247198842.0,-327541732.0,-842910362.0,-1.553529e+09,-76485286.0,-39293328.0,True
2,2403001.0,0.0,-273326486.0,-249414778.0,-330582162.0,-848385458.0,-1.564378e+09,-75834742.0,-39265254.0,True
3,2406001.0,0.0,-274259644.0,-243100592.0,-332130238.0,-868690592.0,-1.547108e+09,-77460044.0,-39589544.0,True
4,2409001.0,0.0,-274359650.0,-243435814.0,-327870770.0,-858800212.0,-1.555334e+09,-76668520.0,-39587594.0,True


In [29]:

train, test = df[df['training?']==True], df[df['training?']==False]
train.shape, test.shape

((7519, 10), (821, 10))

In [30]:
features = df.columns[2:9]
features

Index(['delta', 'theta', 'alpha', 'beta1', 'beta2', 'spindle', 'K_complex'], dtype='object')

In [31]:
clf = RandomForestClassifier(n_estimators = 30, n_jobs = 2, random_state = 0)
clf.fit(train[features], train['stage'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [32]:
pred = clf.predict(test[features])


In [33]:
test.head()

,epoch,stage,delta,theta,alpha,beta1,beta2,spindle,K_complex,training?
0,2397001.0,0.0,-267998884.0,-241505484.0,-336368820.0,-877711454.0,-1.553141e+09,-79331910.0,-39087854.0,False
20,2457001.0,0.0,-274718110.0,-252649748.0,-346444090.0,-901281116.0,-1.560913e+09,-80515786.0,-39697554.0,False
27,2478001.0,0.0,-274773866.0,-254197444.0,-351224930.0,-891162158.0,-1.561591e+09,-80644116.0,-39464344.0,False
29,2484001.0,0.0,-275343974.0,-248471534.0,-332497912.0,-870831752.0,-1.555159e+09,-76440830.0,-39677974.0,False
38,2511001.0,0.0,-274975548.0,-255065030.0,-351622612.0,-918736548.0,-1.565329e+09,-82890224.0,-39471988.0,False


In [34]:
pd.crosstab(test['stage'], pred, rownames = ['Actual'], colnames = ['Predicted'])

Predicted,0.0,1.0,2.0,3.0,5.0
Actual,,,,,
0.0,77,1,50,6,4
1.0,4,5,49,3,5
2.0,39,8,310,19,24
3.0,6,1,55,20,4
5.0,6,4,89,3,29


In [17]:
clf.score(test['stage'].reshape(-1, 1), pred.reshape(-1, 1))

AttributeError: 'Series' object has no attribute 'reshape'